In [2]:
#CROPING 


import os
import glob
import nibabel as nib
import numpy as np
from scipy.ndimage import binary_dilation

# Define the paths to the directories containing images and masks
images_path = '/kaggle/input/plznew/img'
masks_path = '/kaggle/input/plznew/mask'

# Get a list of all image and mask files in the directories
image_files = sorted(glob.glob(os.path.join(images_path, '*.nii')))
mask_files = sorted(glob.glob(os.path.join(masks_path, '*.nii')))

# Define the number of pixels to preserve around non-zero pixels
preservation_radius = 5

# Create the output directory if it doesn't exist
output_directory = '/kaggle/working/cropped'
os.makedirs(output_directory, exist_ok=True)

# Process each pair of image and mask files
for image_file, mask_file in zip(image_files, mask_files):
    # Load the 3D medical image and the binary mask
    image = nib.load(image_file).get_fdata()
    mask = nib.load(mask_file).get_fdata()

    # Element-wise multiplication with the binary mask
    masked_image = image * mask

    # Create a preservation mask using binary dilation
    preservation_mask = binary_dilation(mask, iterations=preservation_radius)

    # Blend the preserved region from the original image with the masked region
    preserved_image = masked_image.copy()
    preserved_image[preservation_mask] = image[preservation_mask]

    # Get the filename (without the path) to use for saving
    output_filename = os.path.basename(image_file)

    # Save the resulting image in the output directory
    output_path = os.path.join(output_directory, output_filename)
    nib.save(nib.Nifti1Image(preserved_image, np.eye(4)), output_path)

    print(f"Cropped and preserved image saved to {output_path}")


Cropped and preserved image saved to /kaggle/working/cropped/10000.nii
Cropped and preserved image saved to /kaggle/working/cropped/10109.nii
Cropped and preserved image saved to /kaggle/working/cropped/10180.nii
Cropped and preserved image saved to /kaggle/working/cropped/10252.nii
Cropped and preserved image saved to /kaggle/working/cropped/10385.nii
Cropped and preserved image saved to /kaggle/working/cropped/10494.nii
Cropped and preserved image saved to /kaggle/working/cropped/11748.nii
Cropped and preserved image saved to /kaggle/working/cropped/1201.nii
Cropped and preserved image saved to /kaggle/working/cropped/12039.nii
Cropped and preserved image saved to /kaggle/working/cropped/12102.nii
Cropped and preserved image saved to /kaggle/working/cropped/12114.nii
Cropped and preserved image saved to /kaggle/working/cropped/12402.nii
Cropped and preserved image saved to /kaggle/working/cropped/12674.nii
Cropped and preserved image saved to /kaggle/working/cropped/12840.nii
Cropped

In [3]:
#CONVERT TO FLOAT32



import os
import nibabel as nib
import numpy as np

# Specify the input directory
input_dir = '/kaggle/working/cropped'

# Function to convert NII files to float32
def convert_to_float32(file_path):
    # Load the NII image
    image = nib.load(file_path)
    image_data = image.get_fdata().astype(np.float32)  # Convert to float32

    # Save the updated image with float32 data type
    new_image = nib.Nifti1Image(image_data, image.affine)
    new_image.to_filename(file_path)

# Process and convert each NII image in the input directory
for file_name in os.listdir(input_dir):
    if file_name.endswith('.nii'):
        file_path = os.path.join(input_dir, file_name)
        convert_to_float32(file_path)

print("Conversion to float32 completed.")


Conversion to float32 completed.


In [4]:
#REMOVE BLACK SLICES + WHAT IS THE ONE WITH MOST SLICES 


import os
import nibabel as nib
import numpy as np

# Specify the input directory
input_dir = '/kaggle/working/cropped'

# Function to remove completely dark slices from an image
def remove_dark_slices(image_data):
    # Calculate the mean intensity of each slice
    slice_means = np.mean(image_data, axis=(0, 1))
    
    # Identify dark slices (where the mean intensity is below a threshold)
    threshold = 1.0  # Adjust this threshold as needed
    dark_slice_indices = np.where(slice_means < threshold)[0]
    
    # Remove the dark slices from the image
    cleaned_image_data = np.delete(image_data, dark_slice_indices, axis=2)
    
    return cleaned_image_data, dark_slice_indices

# Process each NII image in the input directory
max_slices = 0
image_with_most_slices = ""

for file_name in os.listdir(input_dir):
    if file_name.endswith('.nii'):
        file_path = os.path.join(input_dir, file_name)
        
        # Load the NII image
        image = nib.load(file_path)
        image_data = image.get_fdata()
        
        # Remove dark slices
        cleaned_image_data, dark_slice_indices = remove_dark_slices(image_data)
        
        # Update the image if it has more slices than the current max
        if cleaned_image_data.shape[2] > max_slices:
            max_slices = cleaned_image_data.shape[2]
            image_with_most_slices = file_name
        
        # Save the cleaned image back
        cleaned_image = nib.Nifti1Image(cleaned_image_data, image.affine)
        cleaned_image.to_filename(file_path)

print(f"Images processed. Image with the most remaining slices: {image_with_most_slices}, Number of Slices: {max_slices}")

Images processed. Image with the most remaining slices: 51033.nii, Number of Slices: 976


In [5]:
#CONVERT TO FLOAT32



import os
import nibabel as nib
import numpy as np

# Specify the input directory
input_dir = '/kaggle/working/cropped'

# Function to convert NII files to float32
def convert_to_float32(file_path):
    # Load the NII image
    image = nib.load(file_path)
    image_data = image.get_fdata().astype(np.float32)  # Convert to float32

    # Save the updated image with float32 data type
    new_image = nib.Nifti1Image(image_data, image.affine)
    new_image.to_filename(file_path)

# Process and convert each NII image in the input directory
for file_name in os.listdir(input_dir):
    if file_name.endswith('.nii'):
        file_path = os.path.join(input_dir, file_name)
        convert_to_float32(file_path)

print("Conversion to float32 completed.")

Conversion to float32 completed.


In [6]:
#NORMALIZE PICS


import os
import nibabel as nib
import numpy as np

# Define the directory where the weighted images are saved
output_directory = '/kaggle/working/cropped'  # Change to your output directory

# List all weighted image files in the directory
weighted_files = [file for file in os.listdir(output_directory) if file.endswith('.nii')]

# Loop through each weighted image file
for weighted_file in weighted_files:
    # Load the weighted CT scan
    weighted_ct_scan = nib.load(os.path.join(output_directory, weighted_file))
    weighted_ct_data = weighted_ct_scan.get_fdata()

    # Normalize voxel values to have mean zero and unit variance
    mean = np.mean(weighted_ct_data)
    std = np.std(weighted_ct_data)
    normalized_weighted_ct_data = (weighted_ct_data - mean) / std

    # Save the normalized result, overwriting the existing file
    normalized_weighted_nii = nib.Nifti1Image(normalized_weighted_ct_data, affine=weighted_ct_scan.affine)
    nib.save(normalized_weighted_nii, os.path.join(output_directory, weighted_file))

In [7]:
#CONVERT TO FLOAT32



import os
import nibabel as nib
import numpy as np

# Specify the input directory
input_dir = '/kaggle/working/cropped'

# Function to convert NII files to float32
def convert_to_float32(file_path):
    # Load the NII image
    image = nib.load(file_path)
    image_data = image.get_fdata().astype(np.float32)  # Convert to float32

    # Save the updated image with float32 data type
    new_image = nib.Nifti1Image(image_data, image.affine)
    new_image.to_filename(file_path)

# Process and convert each NII image in the input directory
for file_name in os.listdir(input_dir):
    if file_name.endswith('.nii'):
        file_path = os.path.join(input_dir, file_name)
        convert_to_float32(file_path)

print("Conversion to float32 completed.")

Conversion to float32 completed.


In [8]:
import os
import nibabel as nib
import numpy as np

# Function to crop and pad a 3D NIfTI image to the target shape
def crop_and_pad_nifti(input_path, output_path, target_shape):
    # Load the NIfTI image
    img = nib.load(input_path)
    data = img.get_fdata().astype(np.float32)  # Convert to float32
    
    # Get the current shape of the image
    current_shape = data.shape

    # Calculate the variance of pixel values along the slice axis
    slice_variances = np.var(data, axis=(0, 1))

    # Identify slices with low variance
    threshold = np.percentile(slice_variances, 10)  # Adjust the percentile as needed
    low_variance_slices = slice_variances < threshold

    # Remove slices with low variance
    data = data[:, :, ~low_variance_slices]

    # Calculate the number of slices to pad or crop
    num_slices_to_pad_or_crop = target_shape[2] - data.shape[2]

    if num_slices_to_pad_or_crop > 0:
        # Padding: add slices with zeros at the end
        pad_width = ((0, 0), (0, 0), (0, num_slices_to_pad_or_crop))
        data = np.pad(data, pad_width, mode='constant')
    elif num_slices_to_pad_or_crop < 0:
        # Cropping: remove excess slices
        data = data[:, :, :target_shape[2]]

    # Create a new NIfTI image with the modified data
    new_img = nib.Nifti1Image(data, img.affine)

    # Save the modified image to the output path
    nib.save(new_img, output_path)

# Specify the directory containing the input NIfTI files
input_directory = '/kaggle/working/cropped'

# Specify the target shape
target_shape = (128, 128, 512)

# Specify the output directory for saving modified images
output_directory = '/kaggle/working/cropped_output'
os.makedirs(output_directory, exist_ok=True)

# Iterate through all files in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith('.nii'):
        input_path = os.path.join(input_directory, filename)
        output_path = os.path.join(output_directory, filename)
        crop_and_pad_nifti(input_path, output_path, target_shape)

# Print a message when the processing is complete
print("Processing complete.")


Processing complete.


In [ ]:
#DELET PICS


import shutil

# Directory path
directory_to_delete = '/kaggle/working/padded_images1'
# Use shutil.rmtree to delete the entire directory and its contents
shutil.rmtree(directory_to_delete)

# Recreate the directory if needed
# os.makedirs(directory_to_delete)  # Uncomment this line if you want to recreate the directory

print(f"Contents of '{directory_to_delete}' have been deleted.")

In [ ]:
#PREDICT 



import os
import numpy as np
import nibabel as nib
from tensorflow import keras

# Define the NII image path
image_path = '/kaggle/working/processed_images4/48901.nii'  # Update with the path to your NII image

# Function to load and preprocess NII image
def load_and_preprocess_image(image_path):
    image = nib.load(image_path).get_fdata().astype(np.float32)
    # Add preprocessing steps if necessary (e.g., resizing, normalization)
    # Ensure the shape matches the input shape expected by your model
    # Add an extra dimension for the channel if necessary
    return image[..., np.newaxis]

# Load and preprocess the NII image
input_image = load_and_preprocess_image(image_path)

# Make predictions using your trained model (assuming the model is already loaded)
predictions = model.predict(np.expand_dims(input_image, axis=0))

# Apply a threshold of 0.5 to get binary results (0 or 1)
binary_predictions = (predictions >= 0.5).astype(int)

# The binary_predictions will be a binary array for each of the 14 criteria
print(binary_predictions)


In [ ]:
#DETECT IMAGES SHAPE


import os
import nibabel as nib

# Specify the directory path
directory_path = '/kaggle/working/padded_images1'

# Get a list of all NII files in the directory
nii_files = [file for file in os.listdir(directory_path) if file.endswith('.nii')]

# Iterate through the NII files and print their shapes
for file_name in nii_files:
    file_path = os.path.join(directory_path, file_name)
    
    # Load the NII image
    image = nib.load(file_path)
    
    # Get the shape of the image data
    image_shape = image.get_fdata().shape
    
    # Print the file name and its shape
    print(f"File: {file_name}, Shape: {image_shape}")


In [ ]:
#TYPE 32, 64



import os
import nibabel as nib

# Directory containing the NIfTI files
directory_path = '/kaggle/working/padded_images1'  # Replace with your directory path

# Get a list of all NIfTI files in the directory
nifti_files = [file for file in os.listdir(directory_path) if file.endswith('.nii')]

# Dictionary to store data types for each file
data_types = {}

# Loop through each NIfTI file and determine its data type
for file_name in nifti_files:
    file_path = os.path.join(directory_path, file_name)

    # Load the NIfTI file
    img = nib.load(file_path)

    # Get the data type of the NIfTI data
    data_type = str(img.get_data_dtype())

    # Store the data type in the dictionary
    data_types[file_name] = data_type

# Print the data types for each file
for file_name, data_type in data_types.items():
    print(f"{file_name}: {data_type}")


In [ ]:
#CROP AND PAD TO MAKE ALL PICS IN CERTAIN SHAPE  


import os
import nibabel as nib
import numpy as np

# Specify the input and output directories
input_dir = '/kaggle/input/croped/cropped'  # Input directory containing NII images
output_dir = '/kaggle/working/processed_images/'  # Output directory for processed images

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Define the target size (128, 128, 512)
target_size = (128, 128, 256)

# Function to resize or pad images to the target size and save them
def resize_or_pad_and_save(image_path, output_dir, target_size):
    # Load the NII image
    image = nib.load(image_path)
    image_data = image.get_fdata().astype(np.float32)  # Ensure float32 data type

    # Check if the image size matches the target size
    if image_data.shape == target_size:
        # If already at the target size, copy the image to the output directory
        destination_path = os.path.join(output_dir, os.path.basename(image_path))
        nib.save(image, destination_path)
    else:
        # Determine whether to resize or pad based on image size
        if image_data.shape[0] >= target_size[0] and image_data.shape[1] >= target_size[1] and image_data.shape[2] >= target_size[2]:
            # Crop or resize the image to the target size
            processed_image_data = image_data[:target_size[0], :target_size[1], :target_size[2]]
        else:
            # Pad the image to the target size
            pad_width = [(0, max(target_size[0] - image_data.shape[0], 0)),
                         (0, max(target_size[1] - image_data.shape[1], 0)),
                         (0, max(target_size[2] - image_data.shape[2], 0))]
            processed_image_data = np.pad(image_data, pad_width, mode='constant', constant_values=0)

        # Create a new Nifti1Image object with the processed data
        processed_image = nib.Nifti1Image(processed_image_data, image.affine)

        # Save the processed image with float32 data type
        destination_path = os.path.join(output_dir, os.path.basename(image_path))
        nib.save(processed_image, destination_path)

# Process and resize/pad each NII image in the input directory
for image_file in os.listdir(input_dir):
    if image_file.endswith('.nii'):
        image_path = os.path.join(input_dir, image_file)
        resize_or_pad_and_save(image_path, output_dir, target_size)

print("Processing completed.")



In [ ]:
#THE MODEL 


import os
import numpy as np
import pandas as pd
import nibabel as nib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define constants
data_dir = '/kaggle/input/croped256/processed_images'
label_file = '/kaggle/input/labeeel/label.csv'
image_shape = (128, 128, 256, 1)  # Add an extra dimension for the channel
num_classes = 14
batch_size = 2
epochs = 30

# Load labels from CSV
labels_df = pd.read_csv(label_file)
series_ids = labels_df['series_id'].tolist()
labels = labels_df.iloc[:, 1:].values.astype(np.float32)

# Function to load and preprocess NII images
def load_and_preprocess_image(series_id):
    image_path = os.path.join(data_dir, f'{series_id}.nii')
    image = nib.load(image_path).get_fdata().astype(np.float32)
    # Add preprocessing steps if necessary (e.g., normalization)
    return image[..., np.newaxis]  # Add an extra dimension for the channel

# Create a custom data generator using tf.data
def data_generator(series_ids, labels, batch_size, shuffle=True):
    num_samples = len(series_ids)
    indices = list(range(num_samples))
    if shuffle:
        np.random.shuffle(indices)

    for start_idx in range(0, num_samples, batch_size):
        end_idx = min(start_idx + batch_size, num_samples)
        batch_indices = indices[start_idx:end_idx]
        batch_series_ids = [series_ids[i] for i in batch_indices]
        batch_labels = labels[batch_indices]
        batch_images = [load_and_preprocess_image(series_id) for series_id in batch_series_ids]
        yield np.array(batch_images), batch_labels

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(series_ids, labels, test_size=0.2, random_state=42)

# Define the multi-label binary classification model
def create_model():
    model = keras.Sequential([
        model = keras.Sequential([
    layers.Input(shape=image_shape),
    layers.Conv3D(16, (3, 3, 3), activation='relu', padding='same'),
    layers.MaxPooling3D((2, 2, 2)),
    layers.Conv3D(32, (3, 3, 3), activation='relu', padding='same'),
    layers.MaxPooling3D((2, 2, 2)),
    layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same'),
    layers.MaxPooling3D((2, 2, 2)),
    layers.Conv3D(128, (3, 3, 3), activation='relu', padding='same'),
    layers.MaxPooling3D((2, 2, 2)),
    layers.Conv3D(256, (3, 3, 3), activation='relu', padding='same'),
    layers.MaxPooling3D((2, 2, 2)),
    layers.Conv3D(512, (3, 3, 3), activation='relu', padding='same'),
    layers.MaxPooling3D((2, 2, 2)),
    layers.Flatten(),
    layers.Dense(1024, activation='relu'),
    layers.Dense(num_classes, activation='sigmoid')
])
  # Sigmoid for multi-label binary classification
    ])
    return model

# Create MirroredStrategy for multi-GPU training
strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

# Create and compile the model within the strategy's scope
with strategy.scope():
    model = create_model()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model using the custom data generator
train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(X_train, y_train, batch_size),
    output_signature=(
        tf.TensorSpec(shape=(None, *image_shape), dtype=tf.float32),
        tf.TensorSpec(shape=(None, num_classes), dtype=tf.float32)
    )
)

val_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(X_val, y_val, batch_size, shuffle=False),
    output_signature=(
        tf.TensorSpec(shape=(None, *image_shape), dtype=tf.float32),
        tf.TensorSpec(shape=(None, num_classes), dtype=tf.float32)
    )
)

model.fit(train_dataset, epochs=epochs, validation_data=val_dataset)

# Evaluate the model
y_pred = model.predict(val_dataset)
y_pred_binary = (y_pred > 0.5).astype(int)

accuracy = accuracy_score(y_val, y_pred_binary)
precision = precision_score(y_val, y_pred_binary, average='micro')
recall = recall_score(y_val, y_pred_binary, average='micro')
f1 = f1_score(y_val, y_pred_binary, average='micro')

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')


In [ ]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define constants
data_dir = '/kaggle/input/croped/processed_images'
label_file = '/kaggle/input/labeeel/label.csv'
image_shape = (128, 128, 512, 1)  # Add an extra dimension for the channel
num_classes = 14
batch_size = 2
epochs = 30

# Load labels from CSV
labels_df = pd.read_csv(label_file)
series_ids = labels_df['series_id'].tolist()
labels = labels_df.iloc[:, 1:].values.astype(np.float32)

# Function to load and preprocess NII images
def load_and_preprocess_image(series_id):
    image_path = os.path.join(data_dir, f'{series_id}.nii')
    image = nib.load(image_path).get_fdata().astype(np.float32)
    # Add preprocessing steps if necessary (e.g., normalization)
    return image[..., np.newaxis]  # Add an extra dimension for the channel

# Create a custom data generator using tf.data
def data_generator(series_ids, labels, batch_size, shuffle=True):
    num_samples = len(series_ids)
    indices = list(range(num_samples))
    if shuffle:
        np.random.shuffle(indices)

    for start_idx in range(0, num_samples, batch_size):
        end_idx = min(start_idx + batch_size, num_samples)
        batch_indices = indices[start_idx:end_idx]
        batch_series_ids = [series_ids[i] for i in batch_indices]
        batch_labels = labels[batch_indices]
        batch_images = [load_and_preprocess_image(series_id) for series_id in batch_series_ids]
        yield np.array(batch_images), batch_labels

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(series_ids, labels, test_size=0.2, random_state=42)

# Define the multi-label binary classification model
def create_model():
    model = keras.Sequential([
        layers.Input(shape=image_shape),
        layers.Conv3D(32, (3, 3, 3), activation='relu' ),
        layers.MaxPooling3D((2, 2, 2) ),
        layers.Conv3D(64, (3, 3, 3), activation='relu'),
        layers.MaxPooling3D((2, 2, 2)),
        layers.Conv3D(128, (3, 3, 3), activation='relu'),
        layers.MaxPooling3D((2, 2, 2)),
        layers.Conv3D(256, (3, 3, 3), activation='relu'),  # Missing closing parenthesis
        layers.MaxPooling3D((2, 2, 2)),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(num_classes, activation='sigmoid')
    ])
    return model

# Create MirroredStrategy for multi-GPU training
strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

# Create and compile the model within the strategy's scope
with strategy.scope():
    model = create_model()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model using the custom data generator
train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(X_train, y_train, batch_size),
    output_signature=(
        tf.TensorSpec(shape=(None, *image_shape), dtype=tf.float32),
        tf.TensorSpec(shape=(None, num_classes), dtype=tf.float32)
    )
)

val_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(X_val, y_val, batch_size, shuffle=False),
    output_signature=(
        tf.TensorSpec(shape=(None, *image_shape), dtype=tf.float32),
        tf.TensorSpec(shape=(None, num_classes), dtype=tf.float32)
    )
)

model.fit(train_dataset, epochs=epochs, validation_data=val_dataset)

# Evaluate the model
y_pred = model.predict(val_dataset)
y_pred_binary = (y_pred > 0.5).astype(int)

accuracy = accuracy_score(y_val, y_pred_binary)
precision = precision_score(y_val, y_pred_binary, average='micro')
recall = recall_score(y_val, y_pred_binary, average='micro')
f1 = f1_score(y_val, y_pred_binary, average='micro')

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')


In [ ]:
import os
import nibabel as nib
import numpy as np

# Function to crop and pad a 3D NIfTI image to the target shape
def crop_and_pad_nifti(input_path, output_path, target_shape):
    # Load the NIfTI image
    img = nib.load(input_path)
    data = img.get_fdata().astype(np.float32)  # Convert to float32
    
    # Get the current shape of the image
    current_shape = data.shape

    # Calculate the variance of pixel values along the slice axis
    slice_variances = np.var(data, axis=(0, 1))

    # Identify slices with low variance
    threshold = np.percentile(slice_variances, 10)  # Adjust the percentile as needed
    low_variance_slices = slice_variances < threshold

    # Remove slices with low variance
    data = data[:, :, ~low_variance_slices]

    # Calculate the number of slices to pad or crop
    num_slices_to_pad_or_crop = target_shape[2] - data.shape[2]

    if num_slices_to_pad_or_crop > 0:
        # Padding: add slices with zeros at the end
        pad_width = ((0, 0), (0, 0), (0, num_slices_to_pad_or_crop))
        data = np.pad(data, pad_width, mode='constant')
    elif num_slices_to_pad_or_crop < 0:
        # Cropping: remove excess slices
        data = data[:, :, :target_shape[2]]

    # Create a new NIfTI image with the modified data
    new_img = nib.Nifti1Image(data, img.affine)

    # Save the modified image to the output path
    nib.save(new_img, output_path)

# Specify the directory containing the input NIfTI files
input_directory = '/kaggle/input/croped/cropped'

# Specify the target shape
target_shape = (128, 128, 512)

# Specify the output directory for saving modified images
output_directory = '/kaggle/working/cropped_output'
os.makedirs(output_directory, exist_ok=True)

# Iterate through all files in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith('.nii'):
        input_path = os.path.join(input_directory, filename)
        output_path = os.path.join(output_directory, filename)
        crop_and_pad_nifti(input_path, output_path, target_shape)

# Print a message when the processing is complete
print("Processing complete.")
